# 学習したモデルのAttentionの可視化

## 初期化

In [ ]:
import pathlib
from pathlib import Path
import torch
import numpy as np
from typing import List

from aiwolfk2b.AttentionReasoningAgent.Modules.RoleEstimationModelPreprocessor import RoleEstimationModelPreprocessor
from aiwolfk2b.AttentionReasoningAgent.Modules.BERTRoleEstimationModel import BERTRoleEstimationModel
from aiwolfk2b.AttentionReasoningAgent.AbstractModules import RoleEstimationResult
from aiwolfk2b.utils.helper import load_default_GameInfo,load_default_GameSetting,load_config
from aiwolf import Role,Agent

current_dir = pathlib.Path().resolve()
#計算に使うdeviceを取得
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
config = load_config("/home/meip-users/work/AI_Wolf/AIWolfK2B/aiwolfk2b/AttentionReasoningAgent/config_inference.ini")
game_info = load_default_GameInfo()
game_setting = load_default_GameSetting()

estimator = BERTRoleEstimationModel(config)
estimator.initialize(game_info,game_setting)
preprocessor:RoleEstimationModelPreprocessor = estimator.preprocessor
labels_list = preprocessor.role_label_list

## 可視化用関数

In [ ]:
def highlight(word, attn):
  html_color = '#%02X%02X%02X' % (255, int(255*(1 - attn)), int(255*(1 - attn)))
  return '<span style="background-color: {}">{}</span>'.format(html_color, word)

# def mk_html(text,truth_label:Role,result:RoleEstimationResult):
#   #最大確率を持つラベルを予測結果とする
#   pred_label = max(result.probs.items(), key=lambda x: x[1])[0]
#   html = f"正解: {truth_label.name}<br>予測: {pred_label.name}<br>"

#   # 文章の長さ分のdarayを宣言
#   attention_weight = result.attention_map

#   seq_len = attention_weight.shape[1]
#   all_attens = np.zeros((seq_len))

#   # for i in range(12):
#   #   all_attens += attention_weight[i, 0, :]
#   all_attens = np.average(attention_weight[:,0,:], axis=0)
#   #最大値を1,最小値を0として正規化
#   min_val = all_attens.min()
#   max_val = all_attens.max()
#   all_attens = (all_attens - min_val) / (max_val - min_val)

#   # #単語ごとにattentionの和を取る
#   # words_all_attens = []
#   # text_tokens:List[str] = estimator.tokenizer.tokenize(text)
#   # word_list = []
#   # counter = 0
#   # for token in text_tokens:
#   #   word_attention = 0
#   #   ids = estimator.tokenizer.encode(token,add_special_tokens=False)
#   #   #1 word分のattentionを足し込む
#   #   for idx in range(counter,counter + len(ids)):
#   #     word_attention+=all_attens[idx]
#   #   counter += len(ids)
#   #   words_all_attens.append(word_attention)
    
#   #   #一つ前と連続するか
#   #   if token.startswith("##"):
#   #     # 単語
#   #     part_word = token[2:]
#   #   else:
#   #     #連続しない場合
#   #     part_word = token
      
#   #   word_list.append(part_word)
      
#   # for word, attn in zip(word_list,words_all_attens):
#   #     html += highlight(word, attn)
  
#   text_ids = estimator.tokenizer.encode(text)
#   for word, attn in zip(text_ids, all_attens):
#     if estimator.tokenizer.convert_ids_to_tokens([word])[0] == "[SEP]":
#       break
#     html += highlight(estimator.tokenizer.convert_ids_to_tokens([word])[0], attn)
  
#   html += "<br><br>"
#   return html

def mk_html(text,truth_label:Role,result:RoleEstimationResult):
  #最大確率を持つラベルを予測結果とする
  pred_label = max(result.probs.items(), key=lambda x: x[1])[0]
  html = f"正解: {truth_label.name}<br>予測: {pred_label.name}<br>"

  # 文章の長さ分のdarayを宣言
  attention_weight = result.attention_map

  seq_len = attention_weight.shape[1]
  all_attens = np.zeros((seq_len))

  all_attens = np.average(attention_weight[:,0,:], axis=0)
  #最大値を1,最小値を0として正規化
  min_val = all_attens.min()
  max_val = all_attens.max()
  all_attens = (all_attens - min_val) / (max_val - min_val)

  #単語ごとにattentionの和を取る
  agg_words =[]
  agg_attens = []
  text_tokens:List[str] = estimator.tokenizer.tokenize(text)

  
  for idx,token in enumerate(text_tokens):
    #print(token)
    #一つ前と連続するか
    if token.startswith("##"):
      # 単語
      agg_words[-1] += token[2:]
      agg_attens[-1] += all_attens[idx+1]
    else:
      #連続しない場合
      agg_words.append(token)
      agg_attens.append(all_attens[idx+1])
    
  for word, attn in zip(agg_words,agg_attens):
      html += highlight(word, attn)
  
  
  html += "<br><br>"
  return html


## 検証用テキストとその回答

In [ ]:
test_inputs = ["""4,1,0,0,1,1,1,0
day1
talk:
もはもは＾－＾
もはもは＾－＾
もは＾－＾占いCO[02]たん◯れした＾－＾
もは＾－＾
もは＾－＾寒い所では…花は枯れてしまうの…；－；
もはよう＾－＾
占い把握＾－＾
[03]たん占い把握＾－＾
占いco[08]○
占い把握＾－＾
占い2把握＾－＾
[04]たんも占いね＾－＾
占い2把握ら＾－＾真狂か真狐とかかのあ＾－＾
人外全潜伏かしら＾－＾狂人はいなさそう？＾－＾
対抗把握しマス＾－＾
役職欠けて狂狐の可能性も＾－＾
>>13狂いない可能性もある＾－＾
対抗把握＾－＾狂>狼狐かな＾－＾
占い2出てるから全潜伏れはないれそ＾－＾真狂めかのあ＾－＾
[04]たんちょっと出方様子見っぽく思えたから真目下がるのあ；－；
狂人いなくて占い欠けの狼狐らったらやばえ；－；
>>17おんその場合は狼か狐が出てるのもあるなって＾－＾
とりま占い先宣言してほしいお＾－＾
今日はグレラン？＾－＾
グレーから柱出てもらう？＾－＾吊りあんもしゆゆうのいけお＾－＾
呪殺ないと真なのかまからん；－；
吊りは狐先に吊らないとら＾－＾
じゃあ[07]たん占う＾－＾
宣言したほうがいいかんじ？＾－＾
吊り余裕は銃殺出してもらえば増えるし対抗占いしてもらいたいかも＾－＾
じゃあ[06]たん行きます
漏れ吊っていいお＾－＾
day2
divine,1,4,HUMAN
talk:""","""4,1,0,0,1,1,1,0
day1
talk:
もはもは＾－＾
もはもは＾－＾
もは＾－＾占いCO[01]たん◯れした＾－＾
もは＾－＾
もは＾－＾寒い所では…花は枯れてしまうの…；－；
もはよう＾－＾
占い把握＾－＾
[02]たん占い把握＾－＾
占いco[07]○
占い把握＾－＾
占い2把握＾－＾
[03]たんも占いね＾－＾
占い2把握ら＾－＾真狂か真狐とかかのあ＾－＾
人外全潜伏かしら＾－＾狂人はいなさそう？＾－＾
対抗把握しマス＾－＾
役職欠けて狂狐の可能性も＾－＾
>>13狂いない可能性もある＾－＾
対抗把握＾－＾狂>狼狐かな＾－＾
占い2出てるから全潜伏れはないれそ＾－＾真狂めかのあ＾－＾
[03]たんちょっと出方様子見っぽく思えたから真目下がるのあ；－；
狂人いなくて占い欠けの狼狐らったらやばえ；－；
>>17おんその場合は狼か狐が出てるのもあるなって＾－＾
とりま占い先宣言してほしいお＾－＾
今日はグレラン？＾－＾
グレーから柱出てもらう？＾－＾吊りあんもしゆゆうのいけお＾－＾
呪殺ないと真なのかまからん；－；
吊りは狐先に吊らないとら＾－＾
じゃあ[06]たん占う＾－＾
宣言したほうがいいかんじ？＾－＾
吊り余裕は銃殺出してもらえば増えるし対抗占いしてもらいたいかも＾－＾
じゃあ[05]たん行きます
漏れ吊っていいお＾－＾
"""]


truth_labels = [Role.WEREWOLF, Role.VILLAGER]
test_inputs= [preprocessor.preprocess_text(raw) for raw in test_inputs]

test_inputs

In [ ]:
from IPython.display import display, HTML
results = estimator.estimate_from_text(test_inputs)
for i,text in enumerate(test_inputs):
  html_output = mk_html(text,truth_labels[i],results[i])
  display(HTML(html_output))

In [ ]:
tokenizer = estimator.tokenizer
tokenizer.tokenize("じゃあ[05]たん行きます") 

In [ ]:
tokenizer.convert_tokens_to_ids(["##あ"])

In [ ]:
tokenizer.encode("じゃあ[05]たん行きます") 

In [ ]:
tokenizer.encode

In [ ]:
tokenizer.convert_tokens_to_string(a)

In [ ]:
tokenizer.encode('じゃ',add_special_tokens=False)
tokenizer.decode(tokenizer.encode('##あ',add_special_tokens=False),skip_special_tokens=True)